In [ ]:
import os
from pathlib import Path
from dotenv import load_dotenv

# medical_bot/experiments → la raíz del repo es el padre
PROJECT_ROOT = Path.cwd().parent
load_dotenv(PROJECT_ROOT / ".env")

# Rutas y nombres
CHROMA_DIR_ENV = os.getenv("CHROMA_DIR", "vector_store/chroma")
CHROMA_PATH = (PROJECT_ROOT / CHROMA_DIR_ENV).resolve()

COLLECTION_NAME = os.getenv("CHROMA_COLLECTION", "med_kb")
EMBED_MODEL = os.getenv("MODEL_EMBED", "text-embedding-3-small")
CHAT_MODEL = os.getenv("MODEL_CHAT", "gpt-4o-mini")

print("PROJECT_ROOT:", PROJECT_ROOT)
print("CHROMA_PATH:", CHROMA_PATH)
print("COLLECTION:", COLLECTION_NAME)
print("EMBED_MODEL:", EMBED_MODEL)
print("CHAT_MODEL:", CHAT_MODEL)

PROJECT_ROOT: c:\Users\natalia.antepazo\OneDrive - Grupo Gimeno\Repos\medical_bot
CHROMA_PATH: C:\Users\natalia.antepazo\OneDrive - Grupo Gimeno\Repos\medical_bot\vector_store\chroma
COLLECTION: medical_kb
EMBED_MODEL: text-embedding-3-small
CHAT_MODEL: gpt-4o-mini


In [2]:
import chromadb
from openai import OpenAI

# Clientes
chroma_client = chromadb.PersistentClient(path=str(CHROMA_PATH))
col = chroma_client.get_collection(COLLECTION_NAME)

oa = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

print("Chunks en colección:", col.count())


def retrieve(question: str, top_k: int = 5):
    # 1) Embedding de la pregunta (lo haces tú para evitar query_texts)
    q_emb = oa.embeddings.create(model=EMBED_MODEL, input=question).data[0].embedding
    
    # 2) Consulta vectorial
    res = col.query(
        query_embeddings=[q_emb],
        n_results=top_k,
        include=["documents", "metadatas", "distances"],
    )
    # 3) Normaliza a lista cómoda
    chunks = []
    for doc, meta, dist in zip(res["documents"][0], res["metadatas"][0], res["distances"][0]):
        chunks.append({
            "text": doc,
            "source": meta.get("source", ""),
            "distance": float(dist),
            "meta": meta
        })
    return chunks


Chunks en colección: 4


In [3]:
def ask_rag(question: str, top_k: int = 5, show_context: bool = True):
    chunks = retrieve(question, top_k=top_k)

    # Construye contexto “citable”
    context = "\n\n".join(
        [f"[{i+1}] SOURCE: {c['source']}\n{c['text']}" for i, c in enumerate(chunks)]
    )

    system = (
        "Eres un asistente de una clínica de urología. "
        "Respondes de forma clara y profesional. "
        "No haces diagnóstico ni prescribes tratamientos personalizados. "
        "Si aparecen signos de alarma, recomiendas acudir a urgencias o llamar a emergencias.\n"
        "Responde SOLO con el CONTEXTO dado. "
        "Si el contexto no contiene la respuesta, dilo y sugiere pedir cita."
    )

    user = (
        f"CONTEXTO:\n{context}\n\n"
        f"PREGUNTA:\n{question}\n\n"
        "INSTRUCCIONES:\n"
        "- Responde en español.\n"
        "- Sé breve y útil.\n"
        "- Si falta información en el contexto, dilo explícitamente.\n"
    )

    resp = oa.chat.completions.create(
        model=CHAT_MODEL,
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": user},
        ],
        temperature=0.2,
    )

    answer = resp.choices[0].message.content.strip()

    if show_context:
        print("=== QUESTION ===")
        print(question)
        print("\n=== ANSWER ===")
        print(answer)
        print("\n=== SOURCES (top-k) ===")
        for i, c in enumerate(chunks, start=1):
            print(f"[{i}] dist={c['distance']:.4f} src={c['source']}")
        print("\n=== CONTEXT (raw) ===")
        print(context[:2000], "..." if len(context) > 2000 else "")

    return {"answer": answer, "chunks": chunks}


In [4]:
ask_rag("¿Qué hace un urólogo?", top_k=5)


=== QUESTION ===
¿Qué hace un urólogo?

=== ANSWER ===
Un urólogo se encarga del estudio y tratamiento del aparato urinario y del aparato reproductor masculino. Si necesitas más información específica sobre los servicios que ofrece, te sugiero pedir una cita.

=== SOURCES (top-k) ===
[1] dist=0.3244 src=docs\sources\red_flag.md
[2] dist=0.3245 src=docs\sources\urology_general.md
[3] dist=0.4262 src=docs\sources\services.md
[4] dist=0.7058 src=docs\sources\clinic_admin.md

=== CONTEXT (raw) ===
[1] SOURCE: docs\sources\red_flag.md
# Información general sobre urología

La urología es la especialidad médica que se encarga del estudio y tratamiento del aparato urinario y del aparato reproductor masculino.

## ¿Cuándo acudir al urólogo?
Es habitual consultar con un especialista en urología ante síntomas como:
- Molestias urinarias persistentes
- Cambios en el patrón de micción
- Dolor lumbar o abdominal relacionado con el sistema urinario
- Dudas sobre salud sexual masculina

Esta informaci

{'answer': 'Un urólogo se encarga del estudio y tratamiento del aparato urinario y del aparato reproductor masculino. Si necesitas más información específica sobre los servicios que ofrece, te sugiero pedir una cita.',
 'chunks': [{'text': '# Información general sobre urología\n\nLa urología es la especialidad médica que se encarga del estudio y tratamiento del aparato urinario y del aparato reproductor masculino.\n\n## ¿Cuándo acudir al urólogo?\nEs habitual consultar con un especialista en urología ante síntomas como:\n- Molestias urinarias persistentes\n- Cambios en el patrón de micción\n- Dolor lumbar o abdominal relacionado con el sistema urinario\n- Dudas sobre salud sexual masculina\n\nEsta información es educativa y no constituye un diagnóstico.',
   'source': 'docs\\sources\\red_flag.md',
   'distance': 0.3244154453277588,
   'meta': {'source': 'docs\\sources\\red_flag.md',
    'chunk_index': 0,
    'doc_type': 'md'}},
  {'text': '# Información general sobre urología\n\nLa uro

In [8]:
ask_rag("A esa hora no puedo ir. Puedo los viernes a las 18", top_k=5)

=== QUESTION ===
A esa hora no puedo ir. Puedo los viernes a las 18

=== ANSWER ===
No tengo información sobre la disponibilidad de citas específicas en la clínica. Te sugiero que llames al 666666666 o utilices el formulario web para solicitar una cita el viernes a las 18:00.

=== SOURCES (top-k) ===
[1] dist=0.6486 src=docs\sources\clinic_admin.md
[2] dist=0.7674 src=docs\sources\services.md
[3] dist=0.7891 src=docs\sources\urology_general.md
[4] dist=0.7891 src=docs\sources\red_flag.md

=== CONTEXT (raw) ===
[1] SOURCE: docs\sources\clinic_admin.md
# Información administrativa de la clínica

## Horarios
La clínica atiende de lunes a viernes de 9:00 a 13:30 y de 16:00 a 19:00.

## Dirección
La clínica se encuentra en Valencia ciudad. La dirección es calle Mestre Sosa 37

## Solicitud de cita
Las citas pueden solicitarse por teléfono en el 666666666 o a través del formulario web.

## Urgencias
Si el paciente presenta síntomas graves, debe acudir a urgencias hospitalarias.

[2] SOURCE: 

{'answer': 'No tengo información sobre la disponibilidad de citas específicas en la clínica. Te sugiero que llames al 666666666 o utilices el formulario web para solicitar una cita el viernes a las 18:00.',
 'chunks': [{'text': '# Información administrativa de la clínica\n\n## Horarios\nLa clínica atiende de lunes a viernes de 9:00 a 13:30 y de 16:00 a 19:00.\n\n## Dirección\nLa clínica se encuentra en Valencia ciudad. La dirección es calle Mestre Sosa 37\n\n## Solicitud de cita\nLas citas pueden solicitarse por teléfono en el 666666666 o a través del formulario web.\n\n## Urgencias\nSi el paciente presenta síntomas graves, debe acudir a urgencias hospitalarias.',
   'source': 'docs\\sources\\clinic_admin.md',
   'distance': 0.648647665977478,
   'meta': {'doc_type': 'md',
    'chunk_index': 0,
    'source': 'docs\\sources\\clinic_admin.md'}},
  {'text': '# Servicios de urología\n\nLa clínica ofrece atención especializada en urología para adultos.\n\n## Servicios habituales\n- Problema

In [9]:
ask_rag("a las 18 un viernes es posible ir?", top_k=5)

=== QUESTION ===
a las 18 un viernes es posible ir?

=== ANSWER ===
La clínica atiende de lunes a viernes de 9:00 a 13:30 y de 16:00 a 19:00, por lo que a las 18:00 un viernes es posible ir. Sin embargo, si necesitas una cita, te recomiendo que la solicites por teléfono al 666666666 o a través del formulario web.

=== SOURCES (top-k) ===
[1] dist=0.6779 src=docs\sources\clinic_admin.md
[2] dist=0.7570 src=docs\sources\services.md
[3] dist=0.7609 src=docs\sources\urology_general.md
[4] dist=0.7609 src=docs\sources\red_flag.md

=== CONTEXT (raw) ===
[1] SOURCE: docs\sources\clinic_admin.md
# Información administrativa de la clínica

## Horarios
La clínica atiende de lunes a viernes de 9:00 a 13:30 y de 16:00 a 19:00.

## Dirección
La clínica se encuentra en Valencia ciudad. La dirección es calle Mestre Sosa 37

## Solicitud de cita
Las citas pueden solicitarse por teléfono en el 666666666 o a través del formulario web.

## Urgencias
Si el paciente presenta síntomas graves, debe acudir a 

{'answer': 'La clínica atiende de lunes a viernes de 9:00 a 13:30 y de 16:00 a 19:00, por lo que a las 18:00 un viernes es posible ir. Sin embargo, si necesitas una cita, te recomiendo que la solicites por teléfono al 666666666 o a través del formulario web.',
 'chunks': [{'text': '# Información administrativa de la clínica\n\n## Horarios\nLa clínica atiende de lunes a viernes de 9:00 a 13:30 y de 16:00 a 19:00.\n\n## Dirección\nLa clínica se encuentra en Valencia ciudad. La dirección es calle Mestre Sosa 37\n\n## Solicitud de cita\nLas citas pueden solicitarse por teléfono en el 666666666 o a través del formulario web.\n\n## Urgencias\nSi el paciente presenta síntomas graves, debe acudir a urgencias hospitalarias.',
   'source': 'docs\\sources\\clinic_admin.md',
   'distance': 0.6779397130012512,
   'meta': {'source': 'docs\\sources\\clinic_admin.md',
    'chunk_index': 0,
    'doc_type': 'md'}},
  {'text': '# Servicios de urología\n\nLa clínica ofrece atención especializada en urolog

In [10]:
ask_rag("Podría ir un sábado?", top_k=5)

=== QUESTION ===
Podría ir un sábado?

=== ANSWER ===
La clínica no atiende los sábados, solo de lunes a viernes. Si necesitas una cita, puedes solicitarla por teléfono o a través del formulario web.

=== SOURCES (top-k) ===
[1] dist=0.6821 src=docs\sources\clinic_admin.md
[2] dist=0.7466 src=docs\sources\services.md
[3] dist=0.7466 src=docs\sources\urology_general.md
[4] dist=0.7466 src=docs\sources\red_flag.md

=== CONTEXT (raw) ===
[1] SOURCE: docs\sources\clinic_admin.md
# Información administrativa de la clínica

## Horarios
La clínica atiende de lunes a viernes de 9:00 a 13:30 y de 16:00 a 19:00.

## Dirección
La clínica se encuentra en Valencia ciudad. La dirección es calle Mestre Sosa 37

## Solicitud de cita
Las citas pueden solicitarse por teléfono en el 666666666 o a través del formulario web.

## Urgencias
Si el paciente presenta síntomas graves, debe acudir a urgencias hospitalarias.

[2] SOURCE: docs\sources\services.md
# Servicios de urología

La clínica ofrece atención 

{'answer': 'La clínica no atiende los sábados, solo de lunes a viernes. Si necesitas una cita, puedes solicitarla por teléfono o a través del formulario web.',
 'chunks': [{'text': '# Información administrativa de la clínica\n\n## Horarios\nLa clínica atiende de lunes a viernes de 9:00 a 13:30 y de 16:00 a 19:00.\n\n## Dirección\nLa clínica se encuentra en Valencia ciudad. La dirección es calle Mestre Sosa 37\n\n## Solicitud de cita\nLas citas pueden solicitarse por teléfono en el 666666666 o a través del formulario web.\n\n## Urgencias\nSi el paciente presenta síntomas graves, debe acudir a urgencias hospitalarias.',
   'source': 'docs\\sources\\clinic_admin.md',
   'distance': 0.682060956954956,
   'meta': {'source': 'docs\\sources\\clinic_admin.md',
    'doc_type': 'md',
    'chunk_index': 0}},
  {'text': '# Servicios de urología\n\nLa clínica ofrece atención especializada en urología para adultos.\n\n## Servicios habituales\n- Problemas de próstata\n- Litiasis urinaria (cálculos re